Not sure why i does not work. the query return the same values for all sites withon the same day. 
One day is kept in single segment

Some explanation may be here. IE druid doesnot suport exact counts
https://druid.apache.org/docs/latest/querying/sql.html#aggregation-functions

In [239]:
from pydruid.db import connect
import pandas as pd

conn = connect(host='druid', port=8082, path='/druid/v2/sql/', scheme='http')
curs = conn.cursor()
query = '''
select extract(DOY from __time) doy , count("Id") c, cast("Id" as integer) "site" 
from "400_sites_2016_2020" where  \'20119-01-01\'< __time and  
__time <\'2019-02-01\' and cast("Id" as integer)  < 10 group by "Id", extract(doy from __time) 
'''
df = pd.read_sql_query(query,conn)
groupedby = df.groupby(['doy'])
print(df["site"].unique())

#print(df)
print(groupedby['c'].apply(pd.Series.tolist).tolist())



DatabaseError: Execution failed on sql: 
select extract(DOY from __time) doy , count("Id") c, cast("Id" as integer) "site" 
from "400_sites_2016_2020" where  '20119-01-01'< __time and  
__time <'2019-02-01' and cast("Id" as integer)  < 10 group by "Id", extract(doy from __time) 

Unknown exception (java.lang.RuntimeException): While invoking method 'public org.apache.calcite.sql2rel.RelFieldTrimmer$TrimResult org.apache.calcite.sql2rel.RelFieldTrimmer.trimFields(org.apache.calcite.rel.core.Project,org.apache.calcite.util.ImmutableBitSet,java.util.Set)'
unable to rollback

In [240]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Heatmap(
                   z=groupedby['c'].apply(pd.Series.tolist).tolist(),
                   x=df["site"].unique(),
                   y=df["doy"].unique(),
                   hoverongaps = False))
fig.show()